Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/monitor-models/data-drift/drift-on-aks.png)

# Monitor data drift on models deployed to Azure Kubernetes Service 

In this tutorial, you will setup a data drift monitor on a toy model that predicts elevation based on a few weather factors which will send email alerts if drift is detected.

## Prerequisites
If you are using an Azure Machine Learning Compute instance, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) first if you haven't already established your connection to the AzureML Workspace.

In [ ]:
# Check core SDK version number
import azureml.core

print('SDK version:', azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Setup training dataset and model

Setup the training dataset and model in preparation for deployment to the Azure Kubernetes Service. 

The next few cells will:
  * get the default datastore and upload the `training.csv` dataset to the datastore
  * create and register the dataset 
  * register the model with the dataset
  
See the `config.py` script in this folder for details on how `training.csv` and `elevation-regression-model.pkl` are created. If you train your model in Azure ML using a Dataset, it will be automatically captured when registering the model from the run. 

In [ ]:
# use default datastore
dstore = ws.get_default_datastore()

# upload weather data
dstore.upload('dataset', 'drift-on-aks-data', overwrite=True, show_progress=False)

In [ ]:
from azureml.core import Dataset

# create dataset 
dset = Dataset.Tabular.from_delimited_files(dstore.path('drift-on-aks-data/training.csv'))
# register dataset
dset = dset.register(ws, 'drift-demo-dataset')
# get the dataset by name from the workspace
dset = Dataset.get_by_name(ws, 'drift-demo-dataset')

In [ ]:
from azureml.core.model import Model

# register the model
model = Model.register(model_path='elevation-regression-model.pkl',
                       model_name='elevation-regression-model.pkl',
                       tags={'area': "weather", 'type': "linear regression"},
                       description='Linear regression model to predict elevation based on the weather',
                       workspace=ws,
                       datasets=[(Dataset.Scenario.TRAINING, dset)]) # need to register the dataset with the model

## Create the inference config

Create the environment and inference config from the `myenv.yml` and `score.py` files. Notice the [Model Data Collector](https://docs.microsoft.com/azure/machine-learning/service/how-to-enable-data-collection) code included in the scoring script. This dependency is currently required to collect model data, but will be removed in the near future as data collection in Azure Machine Learning webservice endpoints is automated.

In [ ]:
from azureml.core import Environment

# create the environment from the yml file 
env = Environment.from_conda_specification(name='deploytocloudenv', file_path='myenv.yml')

In [ ]:
from azureml.core.model import InferenceConfig

# create an inference config, combining the environment and entry script 
inference_config = InferenceConfig(entry_script='score.py', environment=env)

## Create the AKS compute target

Create an Azure Kubernetes Service compute target to deploy the model to. 

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'drift-aks'

# Create the cluster
try:
    aks_target = ws.compute_targets[aks_name]
except KeyError:
    aks_target = ComputeTarget.create(workspace = ws,
                                      name = aks_name,
                                      provisioning_configuration = prov_config)

    # Wait for the create process to complete
    aks_target.wait_for_completion(show_output = True)

## Deploy the model to AKS 

Deploy the model as a webservice endpoint. Be sure to enable the `collect_model_data` flag so that serving data is collected in blob storage for use by the data drift capability.

In [ ]:
from azureml.core.webservice import AksWebservice

deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, collect_model_data=True)
service_name = 'drift-aks-service'

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)

service.wait_for_deployment(True)
print(service.state)

## Run recent weather data through the webservice 

The below cells take the weather data of Florida from 2019-11-20 to 2019-11-12, filter and transform using the same processes as the training dataset, and runs the data through the service.

In [ ]:
# create dataset 
tset = Dataset.Tabular.from_delimited_files(dstore.path('drift-on-aks-data/testing.csv'))

df = tset.to_pandas_dataframe().fillna(0)

X_features = ['latitude', 'longitude', 'temperature', 'windAngle', 'windSpeed']
y_features = ['elevation']

X = df[X_features]
y = df[y_features]

In [ ]:
import json

data = json.dumps({'data': X.values.tolist()})

data_encoded = bytes(data, encoding='utf8')
prediction = service.run(input_data=data_encoded)
print(prediction)

## Create an Azure Machine Learning Compute cluster

The data drift capability needs a compute target for computing drift and other data metrics. 

In [ ]:
from azureml.core.compute import AmlCompute

compute_name = 'cpu-cluster'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2', min_nodes=0, max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

## Wait 10 minutes

From the Model Data Collector, it can take up to (but usually less than) 10 minutes for data to arrive in your blob storage account. Wait 10 minutes to ensure cells below will run.

In [ ]:
import time

time.sleep(600)

## Create and update the data drift object

In [ ]:
from datetime import datetime, timedelta
from azureml.datadrift import DataDriftDetector, AlertConfiguration

services = [service_name]
start = datetime.now() - timedelta(days=2)
feature_list = X_features
alert_config = AlertConfiguration(['user@contoso.com'])

try:
    monitor = DataDriftDetector.create_from_model(ws, model.name, model.version, services, 
                                                  frequency='Day', 
                                                  schedule_start=datetime.utcnow() + timedelta(days=1), 
                                                  alert_config=alert_config, 
                                                  compute_target='cpu-cluster')
except KeyError:
    monitor = DataDriftDetector.get(ws, model.name, model.version)
    
monitor

In [ ]:
# many monitor settings can be updated 
monitor = monitor.update(drift_threshold = 0.1)

monitor

## Run the monitor on today's scoring data

Perform a data drift run on the data sent to the service earlier in this notebook. If you set your email address in the alert configuration and the drift threshold <=0.1 you should recieve an email alert to drift from this run.

Wait for the run to complete before getting the results. 

In [ ]:
now = datetime.utcnow()
target_date = datetime(now.year, now.month, now.day)
run = monitor.run(target_date, services, feature_list=feature_list, compute_target='cpu-cluster')

## Get and view results and metrics

For enterprise workspaces, the UI in the Azure Machine Learning studio can be used. Otherwise, the metrics can be queried in Python and plotted. 

In [ ]:
# The run() API initiates a pipeline run for each service in the services list. 
# Here we retrieve the individual service run to get its output results and metrics. 

child_run = list(run.get_children())[0]
child_run

In [ ]:
child_run.wait_for_completion(wait_post_processing=True)

In [ ]:
results, metrics = monitor.get_output(run_id=child_run.id)

In [ ]:
drift_plots = monitor.show()

## Enable the monitor's pipeline schedule

Turn on a scheduled pipeline which will anlayze the serving dataset for drift. 

In [ ]:
monitor.enable_schedule()

## Next steps

  * See [our documentation](https://aka.ms/datadrift/aks) or [Python SDK reference](https://docs.microsoft.com/python/api/overview/azure/ml/intro)
  * [Send requests or feedback](mailto:driftfeedback@microsoft.com) on data drift directly to the team
  * Please open issues with data drift here on GitHub or on StackOverflow if others are likely to run into the same issue